In [11]:
from pyteomics import fasta
from os import path, listdir
import os
import pickle
from collections import Counter

In [ ]:
!biosaur

In [5]:
!pip install ete3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 8.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273785 sha256=ec0ba674a15559f455f7593f91fbfb87d3ca05e3ce6347a0865fe04ada6ea359
  Stored in directory: /home/lerost/.cache/pip/wheels/e0/90/21/70d48deb2c8017a28d4fb5fc7bcc0a6954c4e967ef5eea88c8
Successfully built ete3
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [61]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()
import operator

In [7]:
path_to_uniprot = '/home/fasta/uniprot_bacteria.fasta'
path_to_uniprot_dbs = '/home/kae-13-1/fasta/bacts_bases_uniprot/'
path_to_swissprot_dbs = '/home/kae-13-1/fasta/bacts_bases_sprot/'

# I. Creating organism-specific databases

## Swiss-Prot + TrEMBL

In [ ]:
%%time
uniprot_taxid_set = set()
for p in fasta.read(path_to_uniprot):
    spec_i = p[0].split('OX=')[-1].split(' ')[0]
    fasta.write([(p[0], p[1])], output = path.join(path_to_uniprot_dbs, '{}.fasta'.format(spec_i)),
                    file_mode = 'a')
    if spec_i not in uniprot_taxid_set:
        uniprot_taxid_set.update([int(spec_i)])

## Swiss-Prot

In [8]:
%%time
swissprot_taxid_set = set()
for p in fasta.read(path_to_uniprot):
    if p[0].startswith('sp'):
        spec_i = p[0].split('OX=')[-1].split(' ')[0]
        # fasta.write([(p[0], p[1])], output = path.join(path_to_swissprot_dbs, '{}.fasta'.format(spec_i)),
        fasta.write([(p[0], p[1])], output = path.join('/home/lerost/Metaproteomics-DirectMS1/Metaproteomics-DirectMS1_close_rep/tmp/', '{}.fasta'.format(spec_i)),
                        file_mode = 'a')
        if spec_i not in swissprot_taxid_set:
            swissprot_taxid_set.update([int(spec_i)])

KeyboardInterrupt: 

CPU times: user 6min 6s, sys: 15.4 s, total: 6min 22s

Wall time: 6min 17s

In [9]:
for el in list(swissprot_taxid_set) :
    print(el)
    break

196620


In [10]:
swissprot_taxid_set = set()
for p in fasta.read(path_to_uniprot):
    if p[0].startswith('sp'):
        spec_i = p[0].split('OX=')[-1].split(' ')[0]
        print(p[0])
        print(p[1])
        print(spec_i)
    if spec_i not in swissprot_taxid_set:
        swissprot_taxid_set.update([int(spec_i)])
        print(swissprot_taxid_set)
        break

sp|Q1RHU4|PYRG_RICBR CTP synthase OS=Rickettsia bellii (strain RML369-C) OX=336407 GN=pyrG PE=3 SV=1
MVRFIFITGGVVSSLGKGLTAASLAMLLQAKGFKVCLRKLDPYLNVDPGTMNPHQHGEVYVTDDGAETDLDLGHYERFTGVPACKFDNITTGAVYSKLLKEERLGNYTGLTVQVIPHVTNIIKDFILSNTKGFDFVLCEIGGTVGDIEGLPFFEAIRQIGNELKSKQCLFIHLTLLPYVKTARELKTKPTQHSVKELRTIGISPNILVCRAERKIAKSEIEKIALFCNIDPEYVIPAIDQKNIYLVPLAYHDYGLDNKVLKFFNLTIAPSKLNRWHDIIERLKESHSKVRIAIIAKYHKLKDAYKSVIEALDHAGIYHKYKVDLVWINAENVTNENISKKLLGVDGILVPGGFGERATEGKILAVNYARTNNIPFFGICLGMQLAAIEIARNLVGLKDAVTEEFKTAGTKIIERISKHDEDFNPSDITIENIKKTMRLGSYTCNLVSDSITANAYGELKISERHRHRYKFNNDFQDIFEKNGVKFSGFSEDKKIVEAIELPKLLWFVGVQFHPEFKSKPFEAHPLFIRFVEAAIKYNKNN
336407
{336407}


## Calculating number of proteins in databases

In [ ]:
%%time
len_fasta_uniprot = {}
for i in uniprot_taxid_set:
    file = path.join(path_to_uniprot_dbs, '{}.fasta'.format(i))
    # For Windows
    if os.name == 'nt':
        n = sum(1 for _ in fasta.read(file))
    # For Linux
    else:
        n = !grep -o 'OX=' $file | wc -l
    len_fasta_uniprot[i] = int(n[0])

In [12]:
fasta_path = '/home/lerost/Metaproteomics-DirectMS1/Metaproteomics-DirectMS1_close_rep/tmp/100226.fasta'

In [44]:
%%time
n = sum(1 for _ in fasta.read(fasta_path))

CPU times: user 9.35 ms, sys: 0 ns, total: 9.35 ms
Wall time: 8.78 ms


In [45]:
print(n)

804


In [60]:
%%time
n = !grep -o 'OX=' $fasta_path | wc -l

CPU times: user 1.09 ms, sys: 3.07 ms, total: 4.16 ms
Wall time: 19.4 ms


In [47]:
print(n)

['804']


In [ ]:
%%time
len_fasta_sprot = {}
for i in swissprot_taxid_set:
    file = path.join(path_to_swissprot_dbs, '{}.fasta'.format(i))
    # For Windows
    if os.name == 'nt':
        n = sum(1 for _ in fasta.read(file))
    # For Linux
    else:
        n = !grep -o 'OX=' $file | wc -l
    len_fasta_sprot[i] = int(n[0])

In [ ]:
path_to_len_uniprot = ''
path_to_len_swissprot = ''

pickle.dump(len_fasta_sprot, 
            open(path_to_len_swissprot, 'wb'), 
            protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(len_fasta_uniprot, 
            open(path_to_len_uniprot, 'wb'), 
            protocol=pickle.HIGHEST_PROTOCOL)

# II. Creating species:descendants dictionary

In [ ]:
%%time
allowed_ranks = ('strain', 'subspecies', 'forma specialis', 'isolate', 'serotype', 'serogroup')

species_descendants = {}
used = set()

for i in taxid_set:
    if i not in used:
        rank = ncbi.get_rank([i])
        if rank:
            if rank[i] == 'species' :
                descendants = ncbi.get_descendant_taxa(i) + [i]
                descendants = [j for j in descendants if j in taxid_set]
                species_descendants[i] = set(descendants)
                used.update(descendants)
            elif rank[int(i)] in allowed_ranks:
                lineage = ncbi.get_lineage(i)
                ranks = ncbi.get_rank(lineage)
                species = [k for k in ranks.keys() if ranks[k] == 'species'][0]
                
                descendants = ncbi.get_descendant_taxa(species) + [species]
                descendants = [j for j in descendants if j in taxid_set]
                species_descendants[species] = set(descendants)
                used.update(descendants)

In [65]:
%%time
taxid_set = set([100226])
allowed_ranks = ('strain', 'subspecies', 'forma specialis', 'isolate', 'serotype', 'serogroup')

species_descendants = {}
used = set()

for i in taxid_set:
    if i not in used:
        rank = ncbi.get_rank([i])
        if rank:
            if rank[i] == 'species' :
                descendants = ncbi.get_descendant_taxa(i) + [i]
                descendants = [j for j in descendants if j in taxid_set]
                species_descendants[i] = set(descendants)
                used.update(descendants)
            elif rank[int(i)] in allowed_ranks:
                lineage = ncbi.get_lineage(i)
                ranks = ncbi.get_rank(lineage)
                species = [k for k in ranks.keys() if ranks[k] == 'species'][0]
                descendants = ncbi.get_descendant_taxa(species) + [species]
                descendants = [j for j in descendants if j in taxid_set]
                species_descendants[species] = set(descendants)
                used.update(descendants)

CPU times: user 150 ms, sys: 41.1 ms, total: 191 ms
Wall time: 166 ms


In [72]:
# i = 100226
# rank = ncbi.get_rank([i])
print(rank)
print(ncbi.get_descendant_taxa(species) + [species])
print(species_descendants)
print(used)
print(lineage)
print(ranks)
print(species)

{100226: 'strain'}
[100226, 1902]
{1902: {100226}}
{100226}
[1, 131567, 2, 1783272, 201174, 1760, 85011, 2062, 1883, 1477431, 1902, 100226]
{1: 'no rank', 2: 'superkingdom', 1760: 'class', 1883: 'genus', 1902: 'species', 2062: 'family', 85011: 'order', 100226: 'strain', 131567: 'no rank', 201174: 'phylum', 1477431: 'species group', 1783272: 'clade'}
1902


CPU times: user 10h 11s, sys: 1h 56min 50s, total: 11h 57min 2s

Wall time: 11h 58min 20s

In [ ]:
path_to_species_descendants = ''
pickle.dump(species_descendants, 
            open(path_to_species_descendants, 'wb'), 
            protocol=pickle.HIGHEST_PROTOCOL)

# III. Choosing group leaders

## Swiss-Prot + TrEMBL

In [ ]:
%%time
species_leader_uniprot = {}

for i in species_descendants.keys():
    strains = species_des_dict[i]
    lens = {j:len_fasta_uniprot[j] for j in strains} 
    if len(lens) == 0:
        continue
    else:
        lead = max(lens.items(), key=operator.itemgetter(1))[0]
        species_leader_uniprot[i] = lead

## Swiss-Prot

In [94]:
%%time
species_leader_sprot = {}
for i in species_descendants.keys():
    strains = species_des_dict[i]
    strains = [i for i in strains if i in swissprot_taxid_set]
    lens = {j:len_fasta_sprot[j] for j in strains}
    if len(lens) == 0:
        continue
    else:
        lead = max(lens.items(), key=operator.itemgetter(1))[0]
        species_leader_sprot[i] = lead

CPU times: user 28.4 s, sys: 1min 14s, total: 1min 42s
Wall time: 1min 42s


In [ ]:
path_to_leaders_uniprot = ''
path_to_leaders_swissprot = ''
pickle.dump(species_leader_uniprot, 
            open(path_to_leaders_uniprot, 'wb'), 
            protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(species_leader_sprot, 
            open(path_to_leaders_swissprot, 'wb'), 
            protocol=pickle.HIGHEST_PROTOCOL)

In [116]:
lens = {'a':10, 'b':5, 'c':60, 'd':100, 'e':15, 'f':50, 'g':55, 'h':54}

In [131]:
%%timeit
lead = max(lens.items(), key=operator.itemgetter(1))[0]

634 ns ± 2.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [132]:
print(lead)

c


# IV Excluding 'sp' and 'uncultured' organisms

In [99]:
%%time
i = 0
exclude_names = set()
leaders = set(species_leader_sprot.values()).union(set(species_leader_uniprot.values()))

for k in leaders:
    name = list(ncbi.get_taxid_translator([k]).values())[0]
    if 'sp.' in name:
        if name.split(' ')[1] == 'sp.':
            exclude_names.update([k])
            i+=1
    if name.startswith('uncultured'):
        exclude_names.update([k])

CPU times: user 2.02 s, sys: 695 ms, total: 2.71 s
Wall time: 3 s


In [ ]:
path_to_exclude_names = ''
pickle.dump(exclude_names, 
            open(path_to_exclude_names, 'wb'), 
            protocol=pickle.HIGHEST_PROTOCOL)